In [12]:
import numpy as np
import pandas as pd

In [13]:
ratings_set_df = pd.read_csv("./data_movie_lens_100k/ratings_all_development_set.csv")
users_df = pd.read_csv("./data_movie_lens_100k/user_info.csv")
movies_df = pd.read_csv("./data_movie_lens_100k/movie_info.csv")

In [14]:
print(ratings_set_df)
print(users_df)
#print(movies_df)

       user_id  item_id  rating
0          772       36       3
1          471      228       5
2          641      401       4
3          312       98       4
4           58      504       5
...        ...      ...     ...
89987      415      813       4
89988      842      120       3
89989      574      505       2
89990      757      472       5
89991      503      204       3

[89992 rows x 3 columns]
     user_id  age  is_male  orig_user_id
0          0   24        1             1
1          1   53        0             2
2          2   23        1             3
3          3   24        1             4
4          4   33        0             5
..       ...  ...      ...           ...
938      938   26        0           939
939      939   32        1           940
940      940   20        1           941
941      941   48        0           942
942      942   22        1           943

[943 rows x 4 columns]


In [33]:
from sklearn.model_selection import train_test_split

ratings_features = ratings_set_df[["user_id", "item_id"]]
ratings = ratings_set_df["rating"]
X_train, X_test, y_train, y_test = train_test_split(ratings_features, ratings)

In [34]:
from surprise import Dataset, NormalPredictor, Reader, SVD, accuracy

svd = SVD(
    n_factors = 10,
    n_epochs=50
)

reader = Reader(
    line_format='user item rating', sep=',',
    rating_scale=(1, 5), skip_lines=1)
ratings_df = Dataset.load_from_df(ratings_set_df[['user_id', 'item_id','rating']], reader=reader)

trainset = ratings_df.build_full_trainset()

# for n_factors in [3]:
#     ## Fit model like our M3
#     model = SVD(n_factors=n_factors)
#     model.fit(trainset)

#     print("global mean:")
#     print(model.trainset.global_mean)
#     print("shape of bias_per_item: ")
#     print(model.bi.shape)
#     print("shape of bias_per_user: ")
#     print(model.bu.shape)
#     print("shape of U (per user vectors): ")
#     print(model.pu.shape)
#     print("shape of V (per item vectors): ")
#     print(model.qi.shape)

#     print("pu:", model.pu)
user_vectors = svd.pu
item_vectors = svd.qi

user_features = users_df[['user_id', 'age', 'is_male']].values
item_features = movies_df[['release_year']].values

user_ids = user_features[:,0]
# print("user_ids:", user_ids)
inner_uids = [trainset.to_inner_uid(uid) for uid in user_ids]
# print("inner_uids:", inner_uids)

concat_users = np.concatenate((user_features, user_vectors[inner_uids]))
item_ids = item_features[:,0]
# print("user_ids:", user_ids)
inner_iids = [trainset.to_inner_iid(iid) for iid in item_ids]
# print("inner_uids:", inner_uids)

'''
features = []
for pair in test_pairs:
    i, j = pair
    if i in svd.users:
        u_i = svd.U[SVD.to_internal_uid(i)]
    else:
        u_i = 0 vector
    #(same for item)
    feature = [u_i, v_j, user_info[i], item_info[j]]
    features.append(feature)
'''

def get_features_for_user(user_id):
    return []

def get_features_for_item(item_id):
    return []

features = []
for i, j in X_train:
    u_i = np.zeros((svd.pu.shape[0]))
    if trainset.knows_user(i):
        u_i = user_vectors[trainset.to_inner_uid(i)]
    m_j = np.zeros((user_vectors.shape[0]))
    if trainset.knows_item(j):
        m_j = item_vectors[trainset.to_inner_iid(j)]
    features.append(np.concatenate((u_i, m_j, get_features_for_user(i), get_features_for_item(j))))
        
user_vectors = np.resize(user_vectors, (item_features.shape[0], user_vectors.shape[1]))
item_vectors = np.resize(item_vectors, (item_features.shape[0], item_vectors.shape[1]))

user_features = np.resize(user_features, (item_features.shape[0], user_features.shape[1]))

print("user vectors shape:", user_vectors.shape)
print("item vectors shape:", item_vectors.shape)
print("user features:", user_features)

print(item_features.shape)

feature_vectors = np.concatenate((user_vectors, item_vectors, user_features, item_features), axis=1)
print("feature vectors shape:", feature_vectors.shape)




AttributeError: 'SVD' object has no attribute 'pu'

In [17]:
from sklearn.model_selection import train_test_split

ratings = ratings_set_df['rating'].values

labels = (ratings >= 4.5).astype(int)

SEED = 12345

X_train, X_test, y_train, y_test = train_test_split(feature_vectors, labels, test_size=0.25, random_state=SEED)

ValueError: Found input variables with inconsistent numbers of samples: [1681, 89992]